<a href="https://colab.research.google.com/github/annakennung/Erkennung-von-Pflanzenkrankheiten/blob/main/Schmerz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/Colab Notebooks/Archiv.zip'
extract_to = '/content/drive/MyDrive/Colab Notebooks/Archiv'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 122.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00


In [ ]:
# Notwendige Bibliotheken importieren
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import os

# Definieren der Bildgröße und der Anzahl der Klassen
IMG_HEIGHT = 256
IMG_WIDTH = 256
NUM_CLASSES = 38

# Pfade zu den Datensätzen (Beispielpfade - bitte anpassen)
train_dir = '/content/drive/MyDrive/Colab Notebooks/Archiv/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Archiv/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
test_dir = '/content/drive/MyDrive/Colab Notebooks/Archiv/test'

# Überprüfen, ob die Verzeichnisse existieren (optional, aber empfohlen)
if not os.path.exists(train_dir):
    print(f"Fehler: Trainingsverzeichnis nicht gefunden: {train_dir}")
    # Hier können Sie eine Fehlermeldung ausgeben oder den Code beenden
    exit()
if not os.path.exists(validation_dir):
    print(f"Fehler: Validierungsverzeichnis nicht gefunden: {validation_dir}")
    exit()
if not os.path.exists(test_dir):
    print(f"Fehler: Testverzeichnis nicht gefunden: {test_dir}")
    exit()


# Bilddaten-Generatoren erstellen
# Für das Training: Augmentierung kann hilfreich sein
##train_image_generator = ImageDataGenerator(
##    rescale=1./255,
##    rotation_range=40,
##    width_shift_range=0.2,
##    height_shift_range=0.2,
##    shear_range=0.2,
##   zoom_range=0.2,
##   horizontal_flip=True,
##   fill_mode='nearest'
##)

# Für Validierung und Test: Nur Rescaling
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

# Datenströme von Verzeichnissen erstellen
train_data_gen = train_image_generator.flow_from_directory(
    directory=train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32, # Batch-Größe anpassen, falls nötig
    class_mode='categorical'
)

val_data_gen = validation_image_generator.flow_from_directory(
    directory=validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='categorical'
)

test_data_gen = test_image_generator.flow_from_directory(
    directory=test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='categorical',
    shuffle=False # Kein Shuffling für die Testauswertung
)

# Konfiguration des Modells
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
#    Conv2D(128, (3, 3), activation='relu'),
#    MaxPooling2D(2, 2),
#    Conv2D(128, (3, 3), activation='relu'),
#    MaxPooling2D(2, 2),
#    Conv2D(38, (3, 3), activation='relu'),
#    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5), # Dropout zur Vermeidung von Overfitting
    Dense(512, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax') # Softmax für Multiklassen-Klassifizierung
])

# Modell kompilieren
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Modell trainieren
EPOCHS = 10 # Anzahl der Epochen anpassen => erstmal 2 Epochen zum testen
history = model.fit(
    train_data_gen,
    steps_per_epoch=train_data_gen.samples // train_data_gen.batch_size,
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=val_data_gen.samples // val_data_gen.batch_size
)

# Modell auf dem Testdatensatz evaluieren
print("Modell auf dem Testdatensatz evaluieren...")
test_loss, test_acc = model.evaluate(test_data_gen)
print(f'Test-Genauigkeit: {test_acc}')

# Graphische Darstellung der Trainingsentwicklung
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()



Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.
Found 33 images belonging to 1 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
  30/2196 ━━━━━━━━━━━━━━━━━━━━ 7:05:04 12s/step - accuracy: 0.0357 - loss: 8.9636

KeyboardInterrupt: 